<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Send post interactions to HubSpot
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/template.ipynb" target="_parent">
<img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/>
</a>

**Tags:** #tool #naas_drivers #naas #scheduler #asset #snippet #automation #ai #analytics

**Author:** [Firstname Lastname](https://www.linkedin.com/)

* First name
* Last name
* Headline
* Job title
* Company
* Number of followers
* any other information as email, phone if provided.

## Input

### Import library

In [1]:
from naas_drivers import linkedin, hubspot
import naas
import pandas as pd

### Setup LinkedIn
👉 <a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [2]:
# Lindekin cookies
LI_AT = "AQEDARqFkcwFOsAYAAABgJqF6JMAAAGB_d5i4U4Aj1M2kZqsKQ5SH8e8WZU3QO8TRwYguHd7-fwnOd-JPlbPv8_DjbOqpInBeChE0Gzo3uDcg85feqYQEbzqweRDc0a9jAYX38Wy8JPzQ2qkzMkq32AJ" #"AQEDARCNSioDe6wmAAABfqF-HR4AAAF-xYqhHlYAtSu7EZZEpFer0UZF-GLuz2DNSz4asOOyCRxPGFjenv37irMObYYgxxxxxxx"
JSESSIONID = "ajax:7185819897423728931"#"ajax:12XXXXXXXXXXXXXXXXX"
post_link = "https://www.linkedin.com/posts/moajjem-hossain_66daysofdata-datascience-machinelearning-activity-6939455124398305281-UgT-?utm_source=linkedin_share&utm_medium=member_desktop_web"

### Setup your HubSpot
👉 Access your [HubSpot API key](https://knowledge.hubspot.com/integrations/how-do-i-get-my-hubspot-api-key)

In [ ]:
HS_API_KEY = 'YOUR_HUBSPOT_API_KEY'

## Model

### Utility Functions

In [7]:
def get_net(df):
    profile_url = df['PROFILE_URL']
    iden = linkedin.profile.get_network(profile_url)
    cols = ['FOLLOWERS_COUNT']
    for col in cols:
        df[col] = iden[col].item()
    
    return df

def get_id(df):
    profile_url = df['PROFILE_URL']
    iden = linkedin.profile.get_identity(profile_url)
    cols = ['FIRSTNAME', 'LASTNAME', 'OCCUPATION']
    for col in cols:
        df[col] = iden[col].item()
    
    return df

def get_con(df):
    profile_url = df['PROFILE_URL']
    iden  = linkedin.profile.get_contact(profile_url)
    cols = ['EMAIL', 'PHONENUMBER']
    for col in cols:
        df[col] = iden[col].item()
    
    return df

def get_res(df):
    profile_url = df['PROFILE_URL']
    iden  = linkedin.profile.get_resume(profile_url).loc[0,:] # Take only the latest info
    iden  = pd.DataFrame(iden).T
    cols = ['TITLE', 'PLACE']
    for col in cols:
        df[col] = iden[col].item()
    
    return df

def get_all_info(df, comment = False):
    if comment:
        com = pd.DataFrame(df['TEXT'])
    df = pd.DataFrame(df['PROFILE_URL'])
    df = df.apply(get_id, axis = 1).apply(get_net, axis = 1).apply(get_res, axis = 1).apply(get_con, axis = 1)
    df.drop('PROFILE_URL', axis = 1, inplace = True)
    if comment:
        df = pd.concat([df, com], axis = 1)    
    return df

### Connect to LinkedIn and get Likes and Comments

In [4]:
df_posts = linkedin.connect(LI_AT, JSESSIONID) # Connect
df_likes = df_posts.post.get_likes(post_url = post_link) # Get Likes
df_comment = df_posts.post.get_comments(post_url = post_link) # Get comments

In [6]:
# Get info for Likes and Comments
info_likes = get_all_info(df_likes)
info_likes

,FIRSTNAME,LASTNAME,OCCUPATION,FOLLOWERS_COUNT,TITLE,PLACE,EMAIL,PHONENUMBER
0,Jérémy,Ravenel,⚡️ Building @naas.ai - open source data platfo...,12824,Chief Executive Officer,naas.ai,jeremy@naas.ai,+3385256997
1,Ajay,Pawar,Quantitative Researcher/ Analyst. Computationa...,5728,Master of Science - MS,Rennes School of Business,None,None


In [ ]:
info_comment = get_all_info(df_comment, comment = True)
info_comment

## Output

### Save result in csv

In [ ]:
info_comment.to_csv('comments.csv', index=False)
info_likes.to_csv('likes.csv', index=False)

### Share your output with naas

In [ ]:
naas.asset.add('comments.csv')
naas.asset.add('likes.csv')

#-> Uncomment the line below to remove your asset
# naas.asset.delete()